<a href="https://colab.research.google.com/github/ta9ya/Curriculum-Vitae-template/blob/master/jr_east.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import torch
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import lightgbm as lgbm

%matplotlib inline

In [ ]:
# torch.__version__

In [ ]:
train_df = pd.read_csv('drive/MyDrive/jr_east/data/train.csv')
test_df = pd.read_csv('drive/MyDrive/jr_east/data/test.csv')
info_df = pd.read_csv('drive/MyDrive/jr_east/data/info.csv')
network_df = pd.read_csv('drive/MyDrive/jr_east/data/network.csv')
submission = pd.read_csv('drive/MyDrive/jr_east/data/sample_submit.csv', header=None)

In [ ]:
train_df.head()

# delayTime par planArrival

## convert planArrival to minute

In [ ]:
class Dataset:
  def __init__(self, df):
    self.df = df
    self.__preset()
  
  def __preset(self):
    self.df.date = pd.to_datetime(self.df.date, format='%Y%m%d')
    self.df['arrival_minute'] = self.df.planArrival.apply(self.convert2minute)
    self.df['dayofweek'] = self.df.date.apply(lambda x: x.dayofweek)
    self.df.lineName = self.df.lineName.astype('category')
    self.df.stopStation = self.df.stopStation.astype('category')
  
  def convert2minute(self, plan_arrival):
    if plan_arrival.find(':') < 0:
      return None

    h, m = map(int, plan_arrival.split(':'))
    return m + h * 60

  def get_input_data(self):
    input_columns = ['lineName', 'directionCode', 'arrival_minute', 'dayofweek', 'stopStation']
    return self.df[input_columns]

  def get_output_data(self):
    output_columns = ['delayTime']
    return self.df[output_columns]


In [ ]:
train_df['arrival_minute'] = train_df.planArrival.apply(convert2minute)

In [ ]:
train_dataset = Dataset(train_df)

## delayTime by arrival munite of Line A

In [ ]:
def delay_by_time(lineName):
  df = train_df[train_df.lineName==lineName]
  for key, grp in df.groupby('trainNo'):
    plt.plot(grp['arrival_minute'], grp['delayTime'], label=key)

In [ ]:
delay_by_time('A')

In [ ]:
delay_by_time('B')

In [ ]:
delay_by_time('C')

In [ ]:
delay_by_time('D')

In [ ]:
df = train_df[train_df.lineName=='A']
df[(600 < df.arrival_minute) & (df.arrival_minute < 1000)]

## add weeekday data

## create model

In [ ]:
train_dataset = Dataset(train_df)

In [ ]:
test_dataset = Dataset(test_df)

In [ ]:
lgb_train = lgbm.Dataset(
    train_dataset.get_input_data(),
    train_dataset.get_output_data()
)

In [ ]:
lgb_params = {
    'objective': 'regression',
    'metric': 'mae'
}

In [ ]:
train_dataset.get_input_data()

In [ ]:
lgb = lgbm.train(lgb_params, lgb_train)

In [ ]:
delay = lgb.predict(test_dataset.get_input_data())

In [ ]:
test_df['predict'] = delay

In [ ]:
test_df[test_df.delayTime.isnull()][['id', 'predict']].to_csv('submission.csv', header=False, index=False)